In [1]:
class Factor(dict):

    def __init__(self, variables, prob_mapping):
        self.variables = variables
        self.update(prob_mapping)

    def __getitem__(self, mapping):
        "Return probability of row as key or filter mapping to obtain row and return its probability."
        if isinstance(mapping, dict):
            mapping = filter_event_values(self.variables, mapping)  # filter if type is dict
        prob = dict.__getitem__(self, mapping)
        return prob

    def pointwise_product(self, other):
        "Multiply two factors, combining their variables."
        variables = list(set(self.variables) | set(other.variables))
        new_mapping = defaultdict(float)  # new_mapping contains union of vars in both factors
        for new_row in all_events(variables):  # new_row is a row in new_mapping
            p_self = self[Evidence(zip(variables, new_row))]
            p_other = other[Evidence(zip(variables, new_row))]
            # prob of new_row is product of compaitible rows in both factors.
            new_mapping[new_row] = p_self * p_other
        return Factor(variables, new_mapping)

    def sum_out(self, var):
        "Make a factor eliminating var by summing over its values."
        remaining_vars = [X for X in self.variables if X != var]
        new_mapping = defaultdict(float)
        for new_row in all_events(remaining_vars):  # new_row is a row in new_mapping
            # prob of new_row is given by sum of prob of compaitable rows in current dist 
            new_mapping[new_row] = sum(self[row] for row in self if matches_evidence(row,
                                       Evidence(zip(remaining_vars, new_row)), self.variables))
        return Factor(remaining_vars, new_mapping)


def matches_evidence(row, evidence, var_ordering):
    "Does the tuple of values for this row agree with the evidence given variable ordering of the tuple?"
    return all(evidence[v] == row[var_ordering.index(v)]
               for v in evidence)


def all_events(variables):
    "All possible events in joint dist of variables"
    return itertools.product(*[var.domain for var in variables])


def filter_event_values(variable_list, event):
    "Filter and reorder event values to only include vars in variable_list in correct order."
    return tuple(event[var] for var in variable_list)

In [4]:
def normalize(dist):
    """Multiply each number by a constant such that the sum is 1.0"""
    if isinstance(dist, dict):
        total = sum(dist.values())
        for key in dist:
            dist[key] = dist[key] / total
            assert 0 <= dist[key] <= 1, "Probabilities must be between 0 and 1."
        return dist
    total = sum(dist)
    return [(n / total) for n in dist]

In [5]:
from collections import defaultdict, Counter
import itertools
import math
import random

class BayesNet(object):
    "Bayesian network: a graph of variables connected by parent links."
     
    def __init__(self): 
        self.variables = [] # List of variables, in parent-first topological sort order
        self.lookup = {}    # Mapping of {variable_name: variable} pairs
            
    def add(self, name, parentnames, cpt):
        "Add a new Variable to the BayesNet. Parentnames must have been added previously."
        parents = [self.lookup[name] for name in parentnames]
        var = Variable(name, cpt, parents)
        self.variables.append(var)
        self.lookup[name] = var
        return self
    
class Variable(object):
    "A discrete random variable; conditional on zero or more parent Variables."
    
    def __init__(self, name, cpt, parents=()):
        "A variable has a name, list of parent variables, and a Conditional Probability Table."
        self.__name__ = name
        self.parents  = parents
        self.cpt      = CPTable(cpt, parents)
        self.domain   = set(itertools.chain(*self.cpt.values())) # All the outcomes in the CPT
                
    def __repr__(self): return self.__name__
    
class ProbDist(Factor):
    """A Probability Distribution is an {outcome: probability} mapping. 
    The values are normalized to sum to 1.
    ProbDist(0.75) is an abbreviation for ProbDist({T: 0.75, F: 0.25})."""
    def __init__(self, mapping=(), **kwargs):
        if isinstance(mapping, float):
            mapping = {T: mapping, F: 1 - mapping}
        self.update(mapping, **kwargs)
        normalize(self)
        
class Evidence(dict): 
    "A {variable: value} mapping, describing what we know for sure."
        
class CPTable(dict):
    "A mapping of {row: ProbDist, ...} where each row is a tuple of values of the parent variables."
    
    def __init__(self, mapping, parents=()):
        """Provides two shortcuts for writing a Conditional Probability Table. 
        With no parents, CPTable(dist) means CPTable({(): dist}).
        With one parent, CPTable({val: dist,...}) means CPTable({(val,): dist,...})."""
        if len(parents) == 0 and not (isinstance(mapping, dict) and set(mapping.keys()) == {()}):
            mapping = {(): mapping}
        for (row, dist) in mapping.items():
            if len(parents) == 1 and not isinstance(row, tuple): 
                row = (row,)
            self[row] = ProbDist(dist)

class Bool(int):
    "Just like `bool`, except values display as 'T' and 'F' instead of 'True' and 'False'"
    __str__ = __repr__ = lambda self: 'T' if self else 'F'
        
T = Bool(True)
F = Bool(False)

In [6]:
traffic_net = (BayesNet()
    .add('Rain', [], 0.1)
    .add('Traffic', ['Rain'], {T: 0.8, F: 0.1})
    .add('Late', ['Traffic'], {T: 0.3, F: 0.1}))

In [13]:
def variable_to_factor(var):
    # As a convention the first variable will be the variable itself
    variables = [var]
    variables.extend(var.parents)
    factor_mapping = {}
    for row in var.cpt:
        for value, prob in var.cpt[row].items():
            factor_mapping[(value,) + row] = prob

    return Factor(variables, factor_mapping)
            

In [14]:
all_factors = [variable_to_factor(variable) for variable in traffic_net.variables]

In [16]:
all_factors

[{(F,): 0.9, (T,): 0.1},
 {(F, F): 0.9, (F, T): 0.19999999999999996, (T, F): 0.1, (T, T): 0.8},
 {(F, F): 0.9, (F, T): 0.7, (T, F): 0.1, (T, T): 0.3}]

Lets find P(Late). Now Rain and Traffic are hidden vars which we need to eliminate. We will first join on R.

In [17]:
rain_factor = all_factors[0]
traffic_factor = all_factors[1]
late_factor = all_factors[2]

In [19]:
join_rain_traffic = rain_factor.pointwise_product(traffic_factor)

In [20]:
join_rain_traffic

{(F, F): 0.81,
 (F, T): 0.019999999999999997,
 (T, F): 0.09000000000000001,
 (T, T): 0.08000000000000002}

Next we sumout R

In [21]:
traffic_net.variables

[Rain, Traffic, Late]

In [22]:
r_summed_out = join_rain_traffic.sum_out(traffic_net.variables[0])

In [24]:
r_summed_out # P(T)

{(F,): 0.8300000000000001, (T,): 0.17000000000000004}

Now lets join on T

In [25]:
join_on_t = r_summed_out.pointwise_product(late_factor)

In [26]:
join_on_t

{(F, F): 0.7470000000000001,
 (F, T): 0.08300000000000002,
 (T, F): 0.11900000000000002,
 (T, T): 0.05100000000000001}

Now lets summout T to get our answer.

In [27]:
ans = join_on_t.sum_out(traffic_net.variables[1])

In [28]:
ans

{(F,): 0.8660000000000001, (T,): 0.13400000000000004}